# CS202: Compiler Construction

## In-class Exercises, Week of 03/20/2023

----

# Part 1: While Loops

## Question 1

Compile the following program to $\mathcal{C}_{if}$:

```
i = 10
sum = 0
while i > 0:
    i = i - 1
    sum = sum + i
print(sum)
```

<u>Output of RCO</u>
```
i = 10
sum = 0
tmp1 = i > 0
while tmp1:
    tmp2 = i - 1
    i = tmp2
    tmp3 = sum + i
    sum = tmp3
    
print(sim)
```
This means we'll have to do something special in rco

<u>Cif Version:</u>
```
start:
    i = 10
    sum = 0
    goto while_test
    
while_test:
    if i > 0 then got while_body else goto continuation
    
while_body:
    i = i - 1
    sum = sum + i
    goto while_test
    
continuation:
    print(sum)
```

## Question 2

Compile the program above into pseudo-x86 assembly.

```
start:
    i = 10
    sum = 0
    jmp while_test
    
while_test:
    cmpq $0, #i
    jqt while_body
    jmp cont
    
while_body:
    subq $1, #i
    addq #i, #sum
    jmp while_test
    
continuation:
    movq #sum, %rdi
    callq print_int
```

## Question 3

Describe the major changes to the compiler, up to *select instructions*.

- <u>No new passes</u>

- <u>Typechecker</u>
    - Add a case for while-loops
        - Condition must be a boolean
        - Statements must be well-typed

- <u>RCO</u>
    - Add a case for while-loop
        - Easy part: run rco_stmts on the body statements of the loop
        - Hard part: condition
            - Problem: tmp vars created by rco_exp end up outside of the loop
            - Solution:
                - Construct brand-new bindings just for the tmp vars associated with the condition
                - Package up resulting Assign statements into a Begin expression
                    - condition_bindings = {}
                    - new_condition_exp = rco_exp(condition, condition_bindings)
                    - create a Begin node with a list of assignment statements for everything in `condition_bindings` and the expression `new_condition_exp`

- <u>Explicate control</u>
    - New Case: While(Begin(condition_stmts, condition_exp), body_stmts)
        - Create a loop-shaped control flow graph
            - Make a new block for the continuation (use create_block)
            - Make a new block for the condition using the label `test_label` (use create_block)
            - Make a new block for the body statements with the continuation `goto test_label` 
            - Can't use create block to construct the final one
        - Two big differences with `if`
            - We need an explicit test block
            - We can't use create_block for every block we make, because we need to create a cycle in the CFG
        - Process:
            - `continuation_label = ` use `create_block` to add `continuation` to the CFG
            - `test_label = gensym("loop_label")`
            - `body_label = ` use `create_block` to add the result of compiling `body_stmts` to the CFG (like the `then` case for `if`). Continuation should be: `[cif.Goto(test_label)]`
            - Compile the test
                - Let continuation be `[cif.If(explicate_exp(condition_exp), cif.Goto(body_label), cif.Goto(continuation_label)`
                - Compile condition_stmts with this continuation
                - `basic_blocks[test_label] = ` result from above
    
            - Return the new continuation `[cif.Goto(test_label)]`
- <u>Select-Instructions</u>
    - No changes

# Part 2: Dataflow Analysis

## Question 4

Perform liveness analysis on the pseudo-x86 program from Question 2.

Attempt: use the approach from Assignment 3. When we find a jump, go do liveness analysis on the target to get its live-before set.

```
start:
    i = 10
    sum = 0        
    jmp while_test 1{/}
    
while_test:
    cmpq $0, #i    
    jqt while_body 6{sum}
    jmp cont       2{/}
    
while_body:
    subq $1, #i
    addq #i, #sum  8{} # gets stuck in a loop trying to calculate the live-before set for while_test
    jmp while_test 7{/}
    
continuation:       5{sum}
    movq #sum, %rdi 4{/}
    callq print_int 3{/}
```

Problem: to compute live-before of while_test, we need live-before of while_body and vice-versa 

## Question 5

Describe the idea of dataflow analysis on cyclic control-flow graphs.

1. Compute the live-after sets for each block without worrying about jumps (assume all live-before sets are empty). This is an under-approximation of the true live-after sets.
2. Update the live-before sets based on the results from #1
3. Run #1 again until the live-after sets don't change at all. This is called a _fixpoint_

## Question 6

Use the dataflow-based approach to perform liveness analysis on the pseudo-x86 program above.



Change to the liveness analysis in the compiler:
- Add a ul_fixpoint function
    - while-loop
        - Make a copy od the current live-after sets
        - Run `ul_block` on each block of the program
        - Exit the while-loop if the live-after sets are the same as the copy
- Initialize live-before sets to be empty for all blocks
- Remove the call to `ul_block` in the jmp case

## Question 7

How do we know the dataflow analysis will stop (i.e. not loop forever)?

## Question 8

What changes are required for the rest of the compiler?

YOUR ANSWER HERE